In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def split_instrument(df,instrument,splitter):
    strings = df[instrument][0].split(splitter)
    df["i1"],df['i2'] = strings[0],strings[1]
    return df

In [83]:
data1 = pd.DataFrame(columns = ['price','instrument'])
data1['price'] = np.random.normal(10,size=1000)
data1['instrument'] = "BTC-USDT"
data1 = split_instrument(data1,"instrument","-")

In [84]:
data2 = pd.DataFrame(columns = ['price','instrument'])
data2['price'] = np.random.normal(10,size=1000)
data2['instrument'] = "ETH-USDT"
data2 = split_instrument(data2,"instrument","-")

In [96]:
data3 = pd.DataFrame(columns = ['price','instrument'])
data3['price'] = np.random.normal(10,size=1000)
data3['instrument'] = "BTC-ETH"
data3 = split_instrument(data3,"instrument","-")
data3['price'] = (data1['price']/data2['price']) -0.01

In [86]:
data = [data1['instrument'][0],data2['instrument'][0],data3['instrument'][0]]

In [87]:
first_pairs = []
second_pairs = []
for i in data:
    if "BTC" in i:
        first_pairs.append(i)
    if "USDT" in i:
        second_pairs.append(i)


In [88]:
def three_pairs(a,b,c):
    difference = a/b - c
    pct_dif = difference / c
    return pct_dif
    

In [97]:
three_pairs(data1['price'],data2['price'],data3['price'])

0      0.010064
1      0.007907
2      0.012741
3      0.008985
4      0.010632
         ...   
995    0.010190
996    0.008573
997    0.010679
998    0.009691
999    0.008659
Name: price, Length: 1000, dtype: float64

In [ ]:
def trade(df,signal):
    while signal>0:
        #sell ETH for USDT -100ETH
        #Buy BTC for USDT +200BTC
        # Sell BTC for ETH direct -100BTC, +100 ETH ; net up 100BTC

In [77]:
#a positive number indicates the price via usdt of B in terms of A is higher than in the A B market
# Therefore, we should take B, convert it to usdt then sellusdt for A.We should then sell A for B in the direct market, and repeat


,price,instrument,i1,i2
0,10.913011,BTC-USDT,BTC,USDT
1,9.602708,BTC-USDT,BTC,USDT
2,10.479483,BTC-USDT,BTC,USDT
3,7.481865,BTC-USDT,BTC,USDT


In [78]:
data2.head(4)

,price,instrument,i1,i2
0,6.980550,ETH-USDT,ETH,USDT
1,8.376340,ETH-USDT,ETH,USDT
2,11.076664,ETH-USDT,ETH,USDT
3,8.902408,ETH-USDT,ETH,USDT


In [79]:
data3.head(4)

,price,instrument,i1,i2
0,12.754485,BTC-ETH,BTC,ETH
1,13.705003,BTC-ETH,BTC,ETH
2,10.011291,BTC-ETH,BTC,ETH
3,9.677894,BTC-ETH,BTC,ETH
